In [1]:
from logging import getLogger

import numpy as np

from graphenator.quench import quench_particles
from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force

logger = getLogger()

Natom = 192
cell = np.diag(np.array([5.0, 5.0, 5.0]))

x = random_box(Natom) @ cell

pot = lambda r, cell: repulsive_potential(r, cell, repul=4, rc=2.0)
dpot = lambda r, cell: -repulsive_force(r, cell, repul=4, rc=2.0)

r_quenched = quench_particles(x, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 4514.986982
         Iterations: 9
         Function evaluations: 249
         Gradient evaluations: 235


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


In [2]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go
from itertools import starmap


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:, 0],
            y=x[:, 1],
            z=x[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

In [5]:
# The gyroid surface and its gradient
from logging import getLogger, basicConfig, INFO

import numpy as np

from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force

from graphenator.surface.gyroid import Gyroid
from graphenator.surface.sphere import Sphere

# from graphenator.surface.psurface import PSurface


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))
# surf = Gyroid(eccentricity=0.0)
surf = Sphere(eccentricity=0.5)

r = random_box(Natom)
x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

In [6]:
from graphenator.quench import quench_particles
import plotly.graph_objects as go

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

         Current function value: 52063.146672
         Iterations: 3
         Function evaluations: 64
         Gradient evaluations: 53


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



In [8]:
from graphenator import draw_yaplot, graphenate

with open(f"gyroid.yap", "w") as file:
    count = 100
    for x, cell, g in graphenate(
        Natom,
        cell,
        pot,
        dpot,
        dt=0.0005,  # 0.005
        T=0.1,
    ):
        print(count)
        file.write(draw_yaplot(x, cell, g))
        count -= 1
        if count == 0:
            break

/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.

INFO:root:packing candid.
INFO:root:[0, 0, 0, 0, 0, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 0, 0, 0, 0] Cycles in the graph
INFO:root:Tempering
INFO:root:packing candid.
INFO:root:[0, 0, 0, 0, 5, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 0, 0, 0, 0] Cycles in the graph


         Current function value: 35105.554759
         Iterations: 6
         Function evaluations: 57
         Gradient evaluations: 45
100
99


INFO:root:packing candid.
INFO:root:[0, 0, 0, 10, 3, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 0, 0, 0, 0] Cycles in the graph
INFO:root:packing candid.
INFO:root:[0, 0, 0, 88, 19, 12, 8] Cycles in the packing


98


INFO:root:packing candid.
INFO:root:[0, 0, 0, 29, 4, 0, 2] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 87, 21, 16, 6] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 129, 37, 18, 8] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 160, 41, 18, 6] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 199, 40, 12, 3] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 229, 35, 12, 2] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 348, 8, 3, 3] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 264, 28, 9, 3] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 138, 49, 21, 7] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 275, 27, 8, 4] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 158, 46, 25, 6] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 171, 45, 22, 5] Cycles in the packing


...

3


INFO:root:[0, 0, 0, 372, 14, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 8, 128, 56, 0] Cycles in the graph
INFO:root:packing candid.


2


INFO:root:[0, 0, 0, 374, 13, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 8, 128, 56, 0] Cycles in the graph


1


In [9]:
from logging import INFO, basicConfig, getLogger, DEBUG

import numpy as np

from graphenator.quench import quench_particles
from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force
from graphenator.surface import Grid, Ticks, GridSurfaceFunc


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))

r = 2

XYZ = np.mgrid[-5:5:11j, -5:5:11j, -5:5:11j]
xticks = Ticks(min=-5, binw=1.0)
yticks = Ticks(min=-5, binw=1.0)
zticks = Ticks(min=-5, binw=1.0)
grid = Grid(
    values=np.max(np.abs(XYZ), axis=0) - r, xticks=xticks, yticks=yticks, zticks=zticks
)

surf = GridSurfaceFunc(grid)


r = random_box(Natom)
x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r @ cell, cell)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r @ cell, cell)

# r_quenched = quench_particles(r, cell, pot, dpot)
# x_quenched = r_quenched @ cell

         Current function value: 8545.382780
         Iterations: 67
         Function evaluations: 729
         Gradient evaluations: 711


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


In [10]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()